In [30]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime



end_date = datetime(year=2024, month=1, day=1)
start_date = datetime(year=2023, month=1, day=1)


ticker = "INFY.NS"


df_single = yf.download(
    tickers=ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    auto_adjust=False,
    progress=False
)

close_prices = df_single["Close"]
print(close_prices.head())

df_single = df_single[['Close']].dropna()

Ticker          INFY.NS
Date                   
2023-01-02  1524.000000
2023-01-03  1522.550049
2023-01-04  1494.800049
2023-01-05  1475.199951
2023-01-06  1448.500000


WE HAVE THE DATA FOR INFY.NS FROM 1 JAN 2023 TO 1 JAN 2024
USING YFINANCE Python library.




DAILY RETURN
Formulae:

Daily_returns=Close[t+1]-Close[t]/Close[t]


In [31]:
df_single['Daily_Return'] = df_single['Close'].pct_change() * 100

LOG RETURN
Formulae:

log_returns=log(Close[t+1]/Close[t])

In [32]:
df_single['Log_Return'] = np.log(df_single['Close'] / df_single['Close'].shift(1))

In [33]:
df_single.dropna(inplace=True)
print(df_single.head())


Price             Close Daily_Return Log_Return
Ticker          INFY.NS                        
Date                                           
2023-01-03  1522.550049    -0.095141  -0.000952
2023-01-04  1494.800049    -1.822600  -0.018394
2023-01-05  1475.199951    -1.311219  -0.013199
2023-01-06  1448.500000    -1.809921  -0.018265
2023-01-09  1484.449951     2.481874   0.024516


PLOTING

In [34]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=["Daily Return (%)", "Log Return"])

fig.add_trace(go.Scatter(x=df_single.index, y=df_single['Daily_Return'],
                         mode='lines', name='Daily Return'),
              row=1, col=1)


In [35]:
fig.add_trace(go.Scatter(x=df_single.index, y=df_single['Log_Return'],
                         mode='lines', name='Log Return'),
              row=2, col=1)


fig.update_layout(height=600, width=900,
                  title_text="Infosys: Daily vs Log Returns",
                  showlegend=True)

fig.show()

HERE ARE THE DAILY AND LOG RETURNS FOR TIMEPERIOD
- Due to small changes in prices both tends to look same, because mathematically
log returns approximate close answers when the differences between consecutive terms are small.



Step 2: Volatility Estimation

Volatility is the degree of variation in a stock's returns over time,we are using  a 14-day rolling volatility.

In [38]:
df_single['Volatility_14D'] = df_single['Daily_Return'].rolling(window=14).std()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_single.index, y=df_single['Close'],
    name='Close Price', yaxis='y1', line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=df_single.index, y=df_single['Volatility_14D'],
    name='14-Day Volatility', yaxis='y2', line=dict(color='orange')
))

fig.update_layout(
    title='Infosys: 14-Day Rolling Volatility vs Stock Price',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Close Price', side='left'),
    yaxis2=dict(title='Volatility', overlaying='y', side='right'),
    width=950, height=500,
    legend=dict(x=0.01, y=0.99)
)

fig.show()